In [ ]:
def get_token_root_dict():
    all_vc_data = pd.read_csv('../data/data-for-ml-models/word-level-info-and-verb-category-data.csv', delimiter='\t')
    return dict(zip(all_vc_data["Token"], all_vc_data["Root"]))

In [ ]:
problems=[]
equations=[]
for fold in range(0,5):
    test_verbs_set = pd.read_csv('../data/data-for-ml-models/test_samples-fold{}_2.csv'.format(fold), delimiter='\t')
    test_verbs_set = test_verbs_set.drop_duplicates(subset=['Problem'])
    test = list(test_verbs_set['Problem'])
    data = pd.read_csv('../data/data-for-ml-models/word-level-info-and-verb-category-data.csv', delimiter='\t')
    data = data.drop_duplicates(subset=['Problem'])
    data = data[data['Problem'].isin(test)]
    eq = list(data['Equation'])
    problems += test
    equations += eq


In [ ]:
dict_ = {}    
for p, e in zip(problems, equations):
    dict_[p]=e

In [ ]:
problems = dict_.keys()
equations = dict_.values()

In [ ]:
token_root_dict = get_token_root_dict()
token_root_dict['₹']='₹'
token_root_dict['आँवले']='आँवले'
token_root_dict['बॉट']='बाँट'
token_root_dict['लाईन']='लाईन'
token_root_dict['भेड़']='भेड़'
token_root_dict['रूमाल']='रूमाल'
token_root_dict['ढाणी']='ढाणी'
token_root_dict['जूस']='जूस'
token_root_dict['बालक-बालिकाएँ']='बालक'

In [ ]:
from isc_tokenizer import Tokenizer
from isc_tagger import Tagger
from isc_parser import Parser

In [ ]:
import codecs
import os
import pandas as pd

In [ ]:
tk = Tokenizer(lang='hin', split_sen=True)
tagger = Tagger(lang='hin')
parser = Parser(lang='hin')

In [ ]:
from sympy import *
X, y, z = symbols('X y z')

In [ ]:
# problems = list(data['Problem'])
# equations = list(data['Equation'] )
answers = []
sentences = []

In [ ]:
idx = 0
for eq, sent in zip(equations, problems):
    try:
        sympy_eq = sympify("Eq(" + eq.replace("=", ",") + ")")
        answer = solve(sympy_eq)
        sent = sent.replace('की.ग्रा.', 'किलोग्राम')
        sentences.append(sent)
        answers.append(answer[0])
    except:
        print("Time based:", idx, eq)
    idx += 1

In [ ]:
# SOURCE
source = []

for i in range(len(sentences)):
    source.insert(i, (sentences[i], answers[i]))

In [ ]:
def check_money(money):
    if money in ["पैसे" ,"पैसा" ,"रुपये" , "कीमत", "क़ीमत", "लागत", "रुपया", "मूल्य"]:
        return "₹"
    return money

In [ ]:
counter = 0
def kartafunc(sent, transferp, transfern, states, sentence_idx):
    karta = 0
    karma = 0
    recepient = 0
    transfer_type = '0'
    backup_transfer_type = '0'
    tagged_sent = tagger.tag(sent)
    for words in range(len(tagged_sent) - 1):
        current_word = tagged_sent[words][0]
        current_tag = tagged_sent[words][1]
        next_word = tagged_sent[words + 1][0]
        next_tag = tagged_sent[words + 1][1]
        # gets subject and object of sentence by identifying case markers
        if current_tag == 'NNP' or current_tag == 'NNPC':
            if next_word == 'ने' and karta == 0:
                karta = current_word
            elif next_word == 'को':
                recepient = current_word
                backup_transfer_type = 't+'
            elif next_word == 'से':
                recepient = current_word
                backup_transfer_type = 't-'
        if current_tag == 'QC':
            if next_tag == 'JJ':
                if tagged_sent[words + 2][1] == 'NN' or tagged_sent[words + 2][1] == 'NNP' or tagged_sent[words + 2][1] == 'NNS' or tagged_sent[words + 2][1] == 'NNPC':
                    karma = next_word + " " + tagged_sent[words + 2][0]
            elif next_tag == 'NN' or next_tag == 'NNP' or next_tag == 'NNPC' or next_tag == 'NNS':
                karma = next_word

        # in case case markers are used as morphemes
        elif current_tag == 'PRP':
            if len(states)> sentence_idx and states[sentence_idx][0] not in ['', 0, '0']:
                current_word = states[sentence_idx][0]
            
            if 'ने' in current_word or 'वह' in current_word:
                karta = current_word
            elif 'को' in current_word or 'उसे' in current_word:
                recepient = current_word
                backup_transfer_type = 't+'
            elif 'से' in current_word:
                recepient = current_word
                backup_transfer_type = 't-'
                

        elif current_tag == 'VM':
            if current_word in transferp:
                transfer_type = 't+'
            elif current_word in transfern:
                transfer_type = 't-'

    karma = check_money(karma)
    list_karakas = list()
    if karta == karma or karta == recepient or karma == recepient:
        return [0, 0, 0, '0']
    list_karakas.append(karta)
    list_karakas.append(karma)
    list_karakas.append(recepient)
    list_karakas.append(transfer_type)
    return list_karakas

In [ ]:
#!/usr/bin/python2
# -*- coding: utf-8 -*-


def update_state(karta, karma, verb_type, value, sampradaan, all_but_last_states):
    valid_states_idx = []
    valid_states_sampradaan_idx = []
    is_karta_present = False
    is_karma_present_1 = False
    is_karma_present_2 = False
    is_sampradaan_present = False
    state_idx_where_karta = 0
    valid_state_idx_where_karma_1 = 0
    valid_state_idx_where_karma_2 = 0
    state_idx_where_sampradaan = 0

    for state in all_but_last_states:
        if state[0].strip() == karta:
            is_karta_present = True
            valid_states_idx.append(int(state_idx_where_karta))
        if state[0].strip() == sampradaan:
            is_sampradaan_present = True
            valid_states_sampradaan_idx.append(int(state_idx_where_sampradaan))
        state_idx_where_karta += 1
        state_idx_where_sampradaan += 1

    if is_karta_present:
        for state_idx in valid_states_idx:
            if all_but_last_states[state_idx][2].strip() == karma:
                is_karma_present_1 = True
                break
            valid_state_idx_where_karma_1 += 1
    if is_sampradaan_present:
        for state_idx in valid_states_sampradaan_idx:
            if all_but_last_states[state_idx][2].strip() == karma:
                is_karma_present_2 = True
                break
            valid_state_idx_where_karma_2 += 1


    if verb_type == 't-':
        if is_karta_present and is_karma_present_1 and is_karma_present_2 and is_sampradaan_present:
            value_karta = float(all_but_last_states[valid_states_idx[valid_state_idx_where_karma_1]][1])
            value_sampradaan = float(all_but_last_states[valid_states_sampradaan_idx[valid_state_idx_where_karma_2]][1])
            all_but_last_states[valid_states_sampradaan_idx[valid_state_idx_where_karma_2]] = [
                sampradaan, value_sampradaan + float(value), karma]
        elif is_karta_present and is_karma_present_1:
            value_karta = float(all_but_last_states[valid_states_idx[valid_state_idx_where_karma_1]][1])
            all_but_last_states[valid_states_idx[valid_state_idx_where_karma_1]] = [
                karta, value_karta - float(value), karma]
            all_but_last_states.append([sampradaan, float(value), karma])
        elif is_karma_present_2 and is_sampradaan_present:
            value_sampradaan = float(all_but_last_states[valid_states_sampradaan_idx[valid_state_idx_where_karma_2]][1])
            all_but_last_states[valid_states_sampradaan_idx[valid_state_idx_where_karma_2]] = [
                sampradaan, value_sampradaan + float(value), karma]
            all_but_last_states.append([karta, -1 * float(value), karma])
        else:
            all_but_last_states.append([karta, -1*float(value), karma])
            all_but_last_states.append([sampradaan, float(value), karma])

    if verb_type == 't+':
        if is_karta_present and is_karma_present_1 and is_karma_present_2 and is_sampradaan_present:
            value_karta = float(all_but_last_states[valid_states_idx[valid_state_idx_where_karma_1]][1])
            value_sampradaan = float(all_but_last_states[valid_states_sampradaan_idx[valid_state_idx_where_karma_2]][1])
            all_but_last_states[valid_states_idx[valid_state_idx_where_karma_1]] = [
                karta, value_karta + float(value), karma]
            all_but_last_states[valid_states_sampradaan_idx[valid_state_idx_where_karma_2]] = [
                sampradaan, value_sampradaan - float(value), karma]
        elif is_karta_present and is_karma_present_1:
            value_karta = float(all_but_last_states[valid_states_idx[valid_state_idx_where_karma_1]][1])
            all_but_last_states[valid_states_idx[valid_state_idx_where_karma_1]] = [
                karta, value_karta + float(value), karma]
            all_but_last_states.append([sampradaan, -1 * float(value), karma])
        elif is_karma_present_2 and is_sampradaan_present:
            value_sampradaan = float(all_but_last_states[valid_states_sampradaan_idx[valid_state_idx_where_karma_2]][1])
            all_but_last_states[valid_states_sampradaan_idx[valid_state_idx_where_karma_2]] = [
                sampradaan, value_sampradaan - float(value), karma]
            all_but_last_states.append([karta, float(value), karma])
        else:
            all_but_last_states.append([karta, float(value), karma])
            all_but_last_states.append([sampradaan, -1*float(value), karma])

    return all_but_last_states

In [ ]:
def finalsent(sent, all_states, is_transfer, change_default_op):
    '''
    FINDING STATE CORRESPONDING TO QUESTION AND GIVING ANSWER
    '''
    container = []
    positive = ['कुल', 'मिलकर', 'मिलाकर', 'अखंडित']
    negative = ['पहले', 'पेहले', 'ज़्यादा', 'ज्यादा', 'तुलना', 
                'कम', "मुकाबले", 'चाहिए',  'अधिक', 'अतिरिक्त', 'शुरुआत', 'शुरू', 'पिछले']
    tagged_sent = tagger.tag(sent)
    found_container = "*"
    found_entity = "*"
    got_adj = ""
    qf_flag = 0
    found_positive_verb = False
    if is_transfer:  # there is a transfer occuring
        for index in range(0, len(tagged_sent) - 1): # avoids question mark
            current_word = tagged_sent[index][0].strip()
            current_tag = tagged_sent[index][1]
            next_word = tagged_sent[index + 1][0].strip()
            next_tag = tagged_sent[index + 1][1]
            if current_tag == 'NNP' or current_tag == 'NNPC' or current_tag == 'PRP':
                container.append(current_word)
            if current_tag == 'JJ':
                if next_tag == 'NN' or next_tag == 'NNP' or next_tag == 'NNS' or next_tag == 'NNPC':
                    if qf_flag == 1: # LOGIC WORKS ONLY WHEN QUANTITY COMES BEFORE ENTITY
                        found_entity = current_word + " " + token_root_dict[next_word.strip()]
                        break
            # NNS not present in Hindi
            if current_tag == 'NN' or current_tag == 'NNP' or current_tag == 'NNS' or current_tag == 'NNPC':
                if qf_flag == 1: # LOGIC WORKS ONLY WHEN QUANTITY COMES BEFORE ENTITY
                    found_entity = token_root_dict[current_word.strip()]
                    break
            elif current_tag == 'QF':
                qf_flag = 1
                # CONTAINER IS THE LAST PROPER NOUN AFTER QUANTITY
                found_container = container[len(container) - 1].strip()
        for state in all_states:
            # have to make it container specific
            state[0] = state[0].strip()
            state[2] = state[2].strip()
            if state[0].endswith(found_container) and state[2].endswith(found_entity):
                return float(state[1])
        
        # ADDING ALL QUANTITIES - kul kitne, kul milakar
        sum = 0
        for state in all_states:
            state[2] = state[2].strip()
            if state[2].endswith(found_entity) or state[2].startswith(found_entity):
                sum = sum + abs(float(state[1]))
        return sum
    else:
        # FINALISE OPERATION + or -
        # taking + to be default op
        operation = '+'
        if change_default_op:
            operation = '-'
        for words in tagged_sent:
            # if I havent already found out negative
            if change_default_op is False and found_positive_verb is False:
                for check in positive:
                    if words[0] == check:
                        operation = '+'
                        found_positive_verb = True
                        break
            # If I havent already found out positive
            if found_positive_verb is False:
                for check in negative:
                    if words[0] == check:
                        operation = '-'
                        break
        
        # ENTITY FINDER IN QUESTION
        for words in tagged_sent:
            if words[0] == "₹":
                found_entity = token_root_dict[words[0].strip()]
                break
            if (words[1] == 'QF' or words[1] == 'WQ'):
                qf_flag = 1
            if words[1] == 'JJ' and qf_flag == 1:
                got_adj = words[1]
            if (words[1] == 'NN' or words[1] == 'NNP' or words[1] == 'NNS' or words[1] == 'NNPC' or words[1] == 'PRP') and qf_flag == 1:
                found_entity = got_adj.strip() + " " + token_root_dict[words[0].strip()]
                break
        
        sum = 0
        entity_found_in_state = False
        found_entity = found_entity.strip()
                    
        # if money
        if found_entity in ["पैसे" ,"पैसा" ,"रुपये" , "कीमत", "क़ीमत", "लागत", "रुपया", "मूल्य"]:
            found_entity = "₹"
        if operation == '-':
            is_first_state = 0
            for state in all_states:
                # object specific
                state[2] = state[2].strip()
                if state[2].endswith(found_entity):
                    if is_first_state == 0:
                        sum = sum + float(state[1])
                        is_first_state = 1
                        entity_found_in_state = True
                    else:
                        sum = sum - float(state[1])

            # cant find object, assume object of first state is the one
            if entity_found_in_state is False:
                found_entity = token_root_dict[all_states[0][2].strip()]
                is_first_state = 0
                for state in all_states:
                    state[2] = state[2].strip()
                    if state[2].endswith(found_entity) or state[2].startswith(found_entity):
                        if is_first_state == 0:
                            sum = sum + float(state[1])
                            is_first_state = 1
                            entity_found_in_state = True
                        else:
                            sum = sum - float(state[1])
            return sum
        else:
            sum = 0
            for state in all_states:
                # object specific
                state[2] = state[2].strip()
                if state[2].endswith(found_entity):
                    sum = sum + float(state[1])
                    entity_found_in_state = True
            # cant find object, assume object of first item is the one
            if entity_found_in_state is False:
                found_entity = all_states[0][2]
                for state in all_states:
                    state[2] = state[2].strip()
                    if state[2].endswith(found_entity) or state[2].startswith(found_entity):
                        sum = sum + float(state[1])
                        entity_found_in_state = True
            return sum

In [ ]:
def get_tags(sep_sentence):
    wp_tags = []
    for j in sep_sentence:
        wp_tags.append(tagger.tag(j))
    return wp_tags

In [ ]:
def some_change(current_word, current_tag, next_word, next_tag, assign, last_seen_proper_noun):
    if current_word[0].isnumeric() and current_tag != 'QC':
        current_tag = 'QC'
    if next_word[0].isnumeric() and next_tag != 'QC':
        next_tag = 'QC'
    if current_tag == 'PRP' and len(assign) != 0:
        current_word = assign[len(assign) - 1][0]
    elif current_tag == 'PRP' and last_seen_proper_noun:
        current_word = last_seen_proper_noun
    return current_word, current_tag, next_word, next_tag

In [ ]:
wpes = []
total = 0
source_length = len(source)
for i in range(source_length):
    wpes.append(source[i])
    total += 1

In [ ]:
def get_verbcat():
    all_vc_data = pd.read_csv('./verb-cat-full/data/all-verb-cat-data.csv', delimiter='\t')
    dfresult = all_vc_data.dropna()
    token_cat = dfresult[['Token', 'VerbCategory']]
    token_cat = token_cat.drop_duplicates()
    neg=list(token_cat.loc[token_cat['VerbCategory'] == '-', 'Token'])
    transferp=list(token_cat.loc[token_cat['VerbCategory'] == 't+', 'Token'])
    transfern=list(token_cat.loc[token_cat['VerbCategory'] == 't-', 'Token'])
    return neg, transferp, transfern

In [ ]:
negative, transferp, transfern = get_verbcat()

In [ ]:
correct = 0
negative, transferp, transfern = get_verbcat()
wrong = []
for wpi, wpe in enumerate(wpes):
    sentences = tk.tokenize(wpe[0])
    wp_tags = get_tags(sentences)
    containers = []
    values = []
    entities = []
    states = [] # Container, Value, Adjective, Object
    is_transfer = False
    change_default_op = False
    subtract_from_total_given = False
    found_comparison = False
    rem_size = 0
    last_seen_proper_noun = None
    
    for sentence_idx in range(len(sentences)):
        current_sent = sentences[sentence_idx]
        current_tagset = wp_tags[sentence_idx]
        qty_found = False
        entity_found = False
        # Stores the adjective assosciated with entity
        last_seen_attribute = ""
        # Stores the required proper noun, which is the first to the right of QC
        last_seen_proper_noun = ""
        for word_idx in range(0, len(current_tagset) - 1):
            current_word = current_sent[word_idx]
            current_tag = current_tagset[word_idx][1]
            next_word = current_sent[word_idx + 1]
            next_tag = current_tagset[word_idx + 1][1]
            
            current_word, current_tag, next_word, next_tag = some_change(current_word, current_tag, next_word, next_tag, states, last_seen_proper_noun)
            if (found_comparison and current_word=="कम") and len(states) > 0:
                # pehle not in first sentence
                change_default_op = True
                subtract_from_total_given = True
            elif current_word == "तुलना" or current_word == "मुकाबले":
                found_comparison = True
                
            # Handles special case: Money
            if current_word == "₹":
                entities.append("₹")
                values.append(next_word)
                concat_string = last_seen_attribute + " " + token_root_dict[current_word.strip()]
                change_sign = False
                for check in negative:                                  # check for negative verbs
                    if check == current_sent[word_idx + 2]: # use verb association instead of this
                        change_sign = True
                        break
                if change_sign:
                    values[len(values) -
                           1] = float(values[len(values) - 1]) * (-1)
                state = [last_seen_proper_noun, values[len(values) - 1], concat_string]
                states.append(state)
                last_seen_attribute = ""
                qty_found = False
                entity_found = False
                continue
            # in case an adjective is there
            if current_tag == 'JJ' and (next_tag == 'NN' or next_tag == 'NNS' or next_tag == 'NNP') and qty_found:
                last_seen_attribute = current_word
                continue
            if current_tag == 'QC':
                if current_word[0].isnumeric():
                    values.append(current_word)
                    qty_found = True
                    # getting the container
                    containers.append(last_seen_proper_noun)
            elif (current_tag == 'NN' or current_tag == 'NNS' or current_tag == 'NNP') and qty_found:
                if (next_word == 'में' or next_word=="पर") and next_tag=="PSP"and qty_found==1:
                    qty_found = False
                    last_seen_proper_noun = current_word
                    continue
                entity_found = True
                current_word = check_money(current_word)
                concat_string = last_seen_attribute + " " + token_root_dict[current_word.strip()]
                entities.append(concat_string)
                change_sign = False
                # what is the need of the following
                if next_tag == 'VM' or next_word == 'नहीं' or next_word in negative:
                    for check in negative:
                        if check == next_word:
                            change_sign = True
                            break
                if change_sign:
                    values[len(values) -
                           1] = float(values[len(values) - 1]) * (-1)
                # storing the final instance as [ container name, value, object ]
                state = [last_seen_proper_noun, values[len(values) - 1], concat_string]
                states.append(state)
                last_seen_attribute = ""
                qty_found = False
            elif current_tag == 'NNP' or current_tag == 'NNPC' or current_tag == 'PRP':
                # stores the last proper noun before the quantifier
                last_seen_proper_noun = current_word
            elif current_tag == 'VM':
                if entity_found==False and qty_found==True and len(states)>0:
                    entity_found = True
                    current_word = check_money(current_word)
                    concat_string = last_seen_attribute + " " + states[len(states)-1][2].strip()
                    entities.append(concat_string)
                    change_sign = False
                    # what is the need of the following
                    if current_word in negative:
                        for check in negative:
                            if check == next_word:
                                change_sign = True
                                break
                    if change_sign:
                        values[len(values) -
                               1] = float(values[len(values) - 1]) * (-1)
                    # storing the final instance as [ container name, value, object ]
                    state = [last_seen_proper_noun, values[len(values) - 1], concat_string]
                    states.append(state)
                    last_seen_attribute = ""
                    qty_found = False
                # check for transfer
                kartas = kartafunc(current_sent, transferp, transfern, states, sentence_idx)
                if kartas[0] != 0 and kartas[1] != 0 and kartas[2] != 0 and kartas[3]!='0':
                    is_transfer = True
                    temp = []
                    for index2 in range(0, len(states) - 1):
                        temp.append(states[index2])
                    if len(values) > 0:
                        states = update_state(
                        kartas[0], kartas[1], kartas[3], values[len(values) - 1], kartas[2], temp)

                # check for final sentence
        if sentence_idx == len(sentences) - 1:
            if len(states) != rem_size and subtract_from_total_given:
                change_default_op = True
            try:
                store_ans = abs(finalsent(current_sent, states,
                                  is_transfer, change_default_op))
            except:
                store_ans = -10000000
            actual_ans = float(wpe[1])
            pr = 0.01
            diff = abs(actual_ans - store_ans)
            if diff <= pr:
                correct += 1
            else:
                wrong.append((wp_tags, states, actual_ans, store_ans))

print(total)
print(correct)
acc = (correct) / (total)
print("ACCURACY ")
print(acc * 100)